In [1]:
import grab_test as gt
import rtde_control
import rtde_receive
from Motor_code import Motors
import UR5_Interface as ur5
import time
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [53]:
# tmat = np.array([[ 0.91574736, -0.35602909,  0.18614527, -0.02848649],
#        [-0.40073104, -0.77640133,  0.48643151,  0.14332736],
#        [ 0.02866033,  0.52004256,  0.85365937,  0.33750796],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

tmat = np.array([[-0.81726729,  0.57194311,  0.07039362,  0.07964675],
       [-0.57121578, -0.78792257, -0.22997946, -0.04747817],
       [-0.07607045, -0.22816464,  0.97064627,  0.32338142],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

mmc = tmat[:3, 3]
grasp_pose = [mmc[1], -mmc[0], mmc[2]]
tmat[:3, 3] = [0, 0, 0]

In [54]:
# Full test
sleepRate = 1.5
try:
    robotIP = "192.168.0.6"
    con = rtde_control.RTDEControlInterface(robotIP)
    rec = rtde_receive.RTDEReceiveInterface(robotIP)
    ur = ur5.UR5_Interface()
    servoPort = "/dev/ttyACM0"
    gripperController = Motors(servoPort)
    gripperController.torquelimit(600) # used to be 600
    gripperController.speedlimit(100)
    ur.gripperController = gripperController
    try:
        ur.c = con
        ur.r = rec
        ur.gripperController = gripperController
    except Exception as e:
        raise(e)
    else:
        print("UR5 Interface Established")
            
    homePose = ur.getPose()
    
    
    dX,dY,dZ = gt.projectToWorldCoords(grasp_pose, ur)
    

    '''
    stopping point 2/15:
    this works so far
    take home pos
    add translation on the gripper frame to the goal x, y
    apply tmat to the goal
    move to goal (translates on x, y and orients at the same time)
    ## does this address the fact that orientation is not achievable in standstill?

    then: take orig Z (from original gripper frame): [0, 0, dZ]
    transform by tmat (the current orientation of the gripper)
    add transformed dZ to the goal position
    move
    works!
    '''
    goal1 = copy.deepcopy(homePose)
    # goal1 = np.array(goal1)
    # goal1[:3, 3] += [dX, dY, 0]
    goal1.t[0] += dX
    goal1.t[1] += dY
    goal1 = np.array(goal1) @ tmat
    ur.moveL(goal1)

    # todo: take currentPose and validate orientation?
    time.sleep(sleepRate)
    goal2 = goal1
    z_off = -0.00
    # posd = [0, 0, dZ - z_off] @ tmat[:3, :3]
    posd = [0, 0, dZ - z_off]
    goal2 = np.array(goal2)
    goal2[:3, 3] += posd
    ur.moveL(goal2)

    # goal2.t[2] += dZ - 0.03
    # goal3 = np.array(goal2) @ tmat
    # ur.moveL(goal3)
    # time.sleep(sleepRate)

    '''
    end of working section
    '''
    
    currentPose = ur.getPose()
    # desiredPose = np.matmul(np.array(currentPose), np.linalg.inv(tmat))
    # pos = np.array(currentPose.t)
    desiredPose = np.matmul(np.array(currentPose), tmat)
    # desiredPose[:3, 3] = pos
    # desiredPose = np.matmul(np.array(currentPose), tmat_full)
    print(f"currentPose: {currentPose}")
    print("Desired Pose: ", desiredPose)
    # ur.moveL(desiredPose)
    
    #gt.moveToObj(grasp_pose, ur) 
    

    print("Done moving to block")
    # ur.closeGripper(9) 
    time.sleep(sleepRate)
    # gt.moveBackFromObj(homePose, ur)
    # gt.grabStrat1(grasp_pose)
    ur.moveL(homePose)
    ur.openGripper()
    ur.c.disconnect()
    ur.r.disconnect()
except Exception as e:
    gripperController.openGripper()
    gripperController.disconnect()
    ur.c.disconnect()
    ur.r.disconnect()
    raise(e)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
UR5 Interface Established
P_goal:
[ 0.02737199 -0.19089339 -0.1750508 ]
currentPose:    0.7789   -0.6105   -0.1435   -0.06476   
  -0.5343   -0.5262   -0.6616   -0.7536    
   0.3283    0.592    -0.736     0.0819    
   0         0         0         1         

Desired Pose:  [[-0.27699265  0.9592426   0.05593478 -0.06476344]
 [ 0.78753369  0.25999084 -0.55874454 -0.75363934]
 [-0.55051409 -0.1107176  -0.82745141  0.08189839]
 [ 0.          0.          0.          1.        ]]
Done moving to block
Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


In [52]:
# Full test
sleepRate = 1.5

mc = [0.018320640872559864, 0.05933741490882, 0.2980767279668537]
grasp_pose = mc
# grasp_pose = [mc[1], -mc[0], mc[2]]

try:
    robotIP = "192.168.0.6"
    con = rtde_control.RTDEControlInterface(robotIP)
    rec = rtde_receive.RTDEReceiveInterface(robotIP)
    ur = ur5.UR5_Interface()
    try:
        ur.c = con
        ur.r = rec
    except Exception as e:
        raise(e)
    else:
        print("UR5 Interface Established")
            
    # homePose = ur.getPose()
    # gt.moveToObj(grasp_pose, ur) 
    # print("Done moving to block")
    # ur.closeGripper(9) 
    # time.sleep(sleepRate)
    # gt.moveBackFromObj(homePose, ur)
    # ur.openGripper()
    # currentPose = ur.getPose()
    # desiredPose = np.matmul(np.array(currentPose), tmat)
    # print(f"currentPose: {currentPose}")
    # print("Desired Pose: ", desiredPose)
    ur.moveL(homePose)

    ur.c.disconnect()
    ur.r.disconnect()
except Exception as e:
    ur.c.disconnect()
    ur.r.disconnect()
    raise(e)

UR5 Interface Established
